![Cross-encoder Re-ranking](assets/cross-encoder_re-ranking.png "Cross-encoder Re-ranking")

In [1]:
from helper_utils import load_chroma, word_wrap, project_embeddings
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
import numpy as np

In [2]:
embedding_function = SentenceTransformerEmbeddingFunction()

chroma_collection = load_chroma(filename='documents/microsoft_annual_report_2022.pdf', collection_name='microsoft_annual_report_2022', embedding_function=embedding_function)
chroma_collection.count()

/home/adhi/retrieval-augmented-generation/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


349

In [3]:
query = "What has been the investment in research and development?"
results = chroma_collection.query(query_texts=query, n_results=10, include=['documents', 'embeddings'])

retrieved_documents = results['documents'][0]

for document in results['documents'][0]:
    print(word_wrap(document))
    print('')

• operating expenses increased $ 1. 5 billion or 14 % driven by
investments in gaming, search and news advertising, and windows
marketing. operating expenses research and development ( in millions,
except percentages ) 2022 2021 percentage change research and
development $ 24, 512 $ 20, 716 18 % as a percent of revenue 12 % 12 %
0ppt research and development expenses include payroll, employee
benefits, stock - based compensation expense, and other headcount -
related expenses associated with product development. research and
development expenses also include third - party development and
programming costs, localization costs incurred to translate software
for international markets, and the amortization of purchased software
code and services content. research and development expenses increased
$ 3. 8 billion or 18 % driven by investments in cloud engineering,
gaming, and linkedin. sales and marketing

competitive in local markets and enables us to continue to attract top
talent from ac

# Re-ranking

In [4]:
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

config.json: 100%|██████████| 794/794 [00:00<00:00, 5.23MB/s]
pytorch_model.bin: 100%|██████████| 90.9M/90.9M [00:23<00:00, 3.79MB/s]
tokenizer_config.json: 100%|██████████| 316/316 [00:00<00:00, 2.17MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 419kB/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<00:00, 843kB/s]


In [5]:
pairs = [[query, doc] for doc in retrieved_documents]
scores = cross_encoder.predict(pairs)
print("Scores:")
for score in scores:
    print(score)

Scores:
0.98693556
2.6445768
-0.2680301
-5.646994
-4.297035
-10.933231
-7.0384274
-7.3246937
-4.6393747
-10.848564


In [6]:
print("New Ordering:")
for o in np.argsort(scores)[::-1]:
    print(o+1)

New Ordering:
2
1
3
5
9
4
7
8
10
6


# Re-ranking with Query Expansion

In [7]:
original_query = "What were the most important factors that contributed to increases in revenue?"
generated_queries = [
    "What were the major drivers of revenue growth?",
    "Were there any new product launches that contributed to the increase in revenue?",
    "Did any changes in pricing or promotions impact the revenue growth?",
    "What were the key market trends that facilitated the increase in revenue?",
    "Did any acquisitions or partnerships contribute to the revenue growth?"
]

In [8]:
queries = [original_query] + generated_queries

results = chroma_collection.query(query_texts=queries, n_results=10, include=['documents', 'embeddings'])
retrieved_documents = results['documents']

In [9]:
# Deduplicate the retrieved documents
unique_documents = set()
for documents in retrieved_documents:
    for document in documents:
        unique_documents.add(document)

unique_documents = list(unique_documents)

In [10]:
pairs = []
for doc in unique_documents:
    pairs.append([original_query, doc])

In [11]:
scores = cross_encoder.predict(pairs)

In [12]:
print("Scores:")
for score in scores:
    print(score)

Scores:
-3.7948635
-10.148885
-10.711212
-1.1369963
-11.079269
-9.807877
-4.818483
-10.042841
-10.083939
-9.918428
-5.27475
-7.917178
-4.3417673
-7.4042087
-9.357723
-4.651888
-6.90209
-7.7540994
-8.505107
-10.000137
-5.1418324
-9.768024
-3.768152


In [13]:
print("New Ordering:")
for o in np.argsort(scores)[::-1]:
    print(o)

New Ordering:
3
22
0
12
15
6
20
10
16
13
17
11
18
14
21
5
9
19
7
8
1
2
4
